# Where to move in? 
### (Toronto vs NYC neiborhoods)

![alt text](https://images.squarespace-cdn.com/content/v1/5d3c9dfcd2e1860001acb51d/1568582007983-1EUG6EDVM54MHIEMYZEU/ke17ZwdGBToddI8pDm48kPfW58HQawIq7Ev7Hj3E5hxZw-zPPgdn4jUwVcJE1ZvWQUxwkmyExglNqGp0IvTJZUJFbgE-7XRK3dMEBRBhUpwZby2dJfdmC6uznuCz-gLnlBU1V3UIa9Xyt9ajd4Zc9pT0nIw4OBGb5aJH_SxB8e0/Toronto+Vs.+NYC "Logo Title Text 1")

## 1. Discussion and Background of the Business Problem:
### Problem Statement: Make long distance moving an easier decision

In these times of globalization is very usual that people moves from one city to another, even from one country to another for professional or academical reasons. For some, this decision is pleasant and a choice, but for some others is really hard and stressful. Leaving all the familiy, friends, traditions behind makes the task much more difficult. However, we could try to make this decision a bit easier by comparing multiple cities and find similarities of their neighborhoods in order to choose the one that will make you feel more comfortable. 

Between 1993 and 2013 USA was in the TOP 5 countries of immigration to Canada and Canada in the TOP 15 countries to USA. For this reason I find it representative to center the analysis in two cities: NYC and Toronto. So if you currently live in East Toronto, which NYC neighborhood would be the most suitable for you to move in? This is the question we will answer at the end of the analysis, but first let me walk you through all the steps of the project in this long journey from NYC to Toronto or viceversa.

## 2. Description of the data used for the analysis:
In this analysis I will use multiple datasets coming from different sources
- From the **UN**   
    *'International migration flows to and from selected countries: The 2015 revision.'* We will use USA and Canada data files only.   
	The dataset contains annual data on the flows of international migrants as recorded by the countries of destination. The data presents both inflows and outflows according to the place of birth, citizenship or place of previous / next residence both for foreigners and nationals. The current version presents data available from 45 countries.  
	source: https://www.un.org/en/development/desa/population/migration/data/empirical2/migrationflows.asp#
    
	
- From **NYU** (IBM downloaded version)  
	*'2014 New York City Neighborhood Names'*  
	To get a list of the NYC neighborhoods with coordinates.  
    source:https://geo.nyu.edu/catalog/nyu_2451_34572  
    source used: https://cocl.us/new_york_dataset
	
    
- From **Wikipedia**  
	*'List of postal codes of Canada: M.'*      
	It contains full list of the neighborhoods around Toronto by Postal Code  
    source: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
	
    
- **Geocoder API**  
	To get the coordinates of Toronto neighborhoods and NYC
    
	
- **Foursquare venues API**  
	Will give us the list of venues for each neighborhood in order to make the comparison between them
    
#### STEPS TO FOLLOW
- First we will need to obtain the coordinates of NYC and Toronto neighborhoods.  
For that purpose we will use *2014 New York City Neighborhood Names*. In case of Toronto we will need an extra step cause coordinates are not available in the Wikipedia *List of postal codes of Canada: M.*. That's how we will use Geocoder to get the coordinates from the Postal Codes given in the list. After that, we will merge Toronto's and NYC coordinates list into the same dataset in order to manage alltogether.  
- With the list of centroids obtained in the previous step, we will run the Foursquare API in order to find all the venues located around those places 
- Afterwards by using the k-means algorithm we will determine the similarities on the neighborhoods
